# import Package

In [1]:
import openslide
import tifffile
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import csv
import math
import json
import random
import shutil
import scipy.misc
import scipy.ndimage
from skimage import io
from skimage import morphology
from skimage import measure
import mahotas.polygon as ploygon_to_mask
import re

from skimage.morphology import remove_small_holes, remove_small_objects
from scipy.ndimage import binary_dilation
from scipy.ndimage.morphology import binary_closing, binary_opening

from lxml import etree
from tqdm import tqdm
from glob import glob
from PIL import Image
import imutils

from util import read_json_as_dict, get_tissue_mask, get_parent_dir_name
from data_util import get_mask_image, get_rec_info_list, remove_orange_peel, resize_with_preserve_rgb_value
from data_util import get_size4mpp, get_wsi_info_read_region

/tmp/ipykernel_571/1176178118.py:22: DeprecationWarning: Please import `binary_closing` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.morphology import binary_closing, binary_opening
/tmp/ipykernel_571/1176178118.py:22: DeprecationWarning: Please import `binary_opening` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.morphology import binary_closing, binary_opening
2024-12-04 20:43:41.521952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 20:43:41.522005: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12

# Define Function

In [2]:
pass

# Setting Config

In [3]:
json_path = "./config.json"
config_dict = read_json_as_dict(json_path)

mpp_standard = config_dict["mpp_standard"]
train_wsi_path = config_dict['train_wsi_path']
test_wsi_path = config_dict['test_wsi_path']
data_common_path = config_dict['data_common_path']

tissue_mask_level = config_dict["tissue_mask_level"]
downsizing_per_level = config_dict["downsizing_per_level"]
resize_scale = downsizing_per_level ** tissue_mask_level
level_0_patch_size = config_dict["level_0_patch_size"]

print(f"mpp_standard: {mpp_standard}")
print(f"train_wsi_path: {train_wsi_path}")
print(f"test_wsi_path: {test_wsi_path}")
print(f"data_common_path: {data_common_path}")
print(f"tissue_mask_level: {tissue_mask_level}")
print(f"downsizing_per_level: {downsizing_per_level}")
print(f"level_0_patch_size: {level_0_patch_size}")

mpp_standard: 0.65
train_wsi_path: ../data/0_source_data/hubmap-kidney-segmentation/train
test_wsi_path: ../data/0_source_data/hubmap-kidney-segmentation/test
data_common_path: ../data
tissue_mask_level: 2
downsizing_per_level: 2
level_0_patch_size: 1024


In [4]:
tissue_mask_folder_basename = "1_tissue_mask"
tissue_mask_folder = f"{data_common_path}/{tissue_mask_folder_basename}"
os.makedirs(tissue_mask_folder, exist_ok=True)

wsi_path_list = glob(f"{train_wsi_path}/*.tiff") + glob(f"{test_wsi_path}/*.tiff")

use_memmap = False

# Get Tissue Mask

In [5]:
for wsi_path in tqdm(wsi_path_list):
    wsi_basename = get_parent_dir_name(wsi_path, level=0)
    tissue_mask_basename = get_parent_dir_name(wsi_path, level=0).replace('.tiff', '_tissue_mask.png')
    tissue_mask_path = f"{tissue_mask_folder}/{tissue_mask_basename}"
    if os.path.exists(tissue_mask_path):
        continue
    wsi_array, mpp_value = get_wsi_info_read_region(wsi_path, resize_scale, use_memmap=use_memmap)
    if wsi_array.shape[-1] == 4:
        wsi_array = wsi_array[..., :3]
    level_0_patch_size_mpp = get_size4mpp(level_0_patch_size, mpp_standard, mpp_value)
    downsize_level_0_patch_size_mpp = int(round(level_0_patch_size_mpp / resize_scale))
    wsi_row, wsi_col, wsi_channel = wsi_array.shape
    
    if wsi_array.shape[-1] != 3:
        wsi_array = wsi_array.transpose(1, 2, 0)
    assert wsi_array.shape[-1] == 3
    # Change Background to White
    background_index = np.logical_or(wsi_array.sum(axis=-1) < 50, wsi_array.sum(axis=-1) > 600)
    wsi_array[background_index] = [255, 255, 255]
    # Replace grayscale pixels with white pixels.
    grayscale_index = (wsi_array.max(axis=-1) - wsi_array.min(axis=-1)) < 20
    wsi_array[grayscale_index] = [255, 255, 255]
    
    wsi_tissue_mask = get_tissue_mask(wsi_array, luminosity_threshold=0.85)
    for i in range(10):
        wsi_tissue_mask = binary_dilation(wsi_tissue_mask)
    
    patch_area = downsize_level_0_patch_size_mpp ** 2
    
    wsi_tissue_mask = remove_small_holes(wsi_tissue_mask, area_threshold=wsi_row * wsi_col // 200)
    wsi_tissue_mask = remove_small_objects(wsi_tissue_mask, min_size=patch_area * 0.1)
    wsi_tissue_mask = np.round(wsi_tissue_mask * 255).astype("uint8")
    Image.fromarray(wsi_tissue_mask).save(tissue_mask_path)


100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [11:59<00:00, 35.96s/it]


# Unused Code